<a href="https://colab.research.google.com/github/rahulhingve/Daily-useful-scripts/blob/main/Download_And_Stream_files_from_Colab_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
# There is 2 seperate script in this file
# 1st script is to download the file in colab /content/DownloadedFiles/
# So download the file first by replacing the url
# 2nd Script is to serve the /content folder to the Public ip so anyone can access and Download the files From Anywhere
# so Sign up to https://ngrok.com/ and get your authtoken first and paste in the 2nd script
# ////////////////////////////////////////////////////
#  
#              Things to Improve
#   Add resumeable Download Support (you cannot pause or resume the link for now this the main cause if you streaming it u cant go forward or backward  )
```



In [ ]:
from urllib.parse import urlparse, unquote
import os
import requests
from tqdm import tqdm

# Define the URL and the target directory in Colab
url = 'https://ddl2.ynunciksuf.workers.dev/1396601676/32d481a254e17cd0a7d3bde9fcd5ce46e62ca76d6b950c0d4f40fa3efc1bcd0c362b2279a09b3fee52ddfa5b0f7a3885c522607f6658d61663f27961432efdae8cf5c0ca98f47d79d74163d2188cd4de6ba2255f279f79f0b0cbc39ef3cb0c7642bca9df4b72150670bf786f67648f5c2bbccc07febf732d4deb26c66b7536b65609d9d8638821dc588441e91d57208a706291fc6b943227d8a362d9e9918b8997b7a96a9fa77838db706ce361634649461493ae1569744009b1288b037d1235072065ac89107ffce3cb308732e9dee3d4bfe5dcf2f88e583f0ae92e2d032ddefc3da6ffc0b5cad1acd845180be16fd8021a8a773dc9c114c3ad67a1d79b83b75d83ab89b652b186d28c3d1e85236cc3d010cf524d9f393379bf6d885e088b51bfe1494bfe4b7e4b96613c028aca70d136fb46657918672ddac53bc8d89b029a0dcff16440429c81522ca1319b4cd046fb93d77ef5f681e5f5c672fd6ea7d84fa3ca5e7f81f898cf49d02c31abd92959300accc208b2cd633b96a76189a2b06bb1c26dc3f5193eca89852abd85de3cac1c052d74cf5a426b249d5783e9543fdfb6768bea71fc6141e4c7cf435ab9fbd5::56893312d24e64bf038ae910453eda4e/Swatantrya.Veer.Savarkar.2024.2160p.ZEE5.WEB-DL.DD+5.1.H.265-Telly.mkv'
target_directory = '/content/DownloadedFiles/'

###################### DONT MESS WITH CODE UNTIL YOU KNOW WHAT ARE YOU DOING    #########################################################################
os.makedirs(target_directory, exist_ok=True)

parsed_url = urlparse(url)
file_name = os.path.basename(unquote(parsed_url.path))

destination_path = os.path.join(target_directory, file_name)

session = requests.Session()

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


response = session.get(url, headers=headers, stream=True)


if response.status_code == 200:

    total_size = int(response.headers.get('content-length', 0))
    with open(destination_path, 'wb') as f, tqdm(
        desc=file_name,
        total=total_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                f.write(chunk)
                bar.update(len(chunk))

    print(f"File has been downloaded to {destination_path}")
else:
    print('Failed to download file. Status code:', response.status_code)


In [ ]:

!pip install pyngrok
from pyngrok import ngrok

import os
from http.server import SimpleHTTPRequestHandler
from socketserver import TCPServer
import threading

# Set ngrok authtoken
authtoken = "PASTE HERE YOUR NGROK AUTH TOKEN "
ngrok.set_auth_token(authtoken)

############### AGAIN DONT MESS WITH CODE ################################
def find_free_port():
    import socket
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        return s.getsockname()[1]


port = find_free_port()

os.chdir('/content')


handler = SimpleHTTPRequestHandler

httpd = TCPServer(('localhost', port), handler)


thread = threading.Thread(target=httpd.serve_forever)
thread.start()

print(f"HTTP server started on port {port}")


public_url = ngrok.connect(addr=port, proto="http")
print(f'Public URL: {public_url}')


try:
    while True:
        pass
except KeyboardInterrupt:
    print("Shutting down the server...")
    httpd.shutdown()
